## Set the path of where the csv files are located, as well as initial import statements

In [9]:
csv_data_path = "/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/collection/formal_scripts/output_data_ocellID_01_2017.csv" # **PLACE CSV FILEPATH HERE**
sep = "\t" # csv delimiter used
small_model = False # use smaller ML model for better performance, less accuracy

In [10]:
from pathlib import Path 
import pandas as pd
import numpy as np

# Gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/quinton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Functions to handle data loading and data writing

In [11]:
def load_data(filename) -> pd.DataFrame:
    filename = Path(filename)
    if not filename.is_file():
        raise FileNotFoundError

    with open(filename.resolve(), 'r', encoding='utf-8') as file:
        data = pd.read_csv(file, sep=sep)
    return data

def load_data_by_year(directory: Path) -> pd.DataFrame:
    for data_file in Path.glob(Path.joinpath(directory, '*.csv')):
            yield load_data(data_file.resolve())

def write_data(filename: Path, data: pd.DataFrame):
    with open(filename.as_uri(), 'w', encoding='utf-8') as f:
        data.to_csv(f, sep=sep)


In [12]:
# obtain the stop words
stopwords = stopwords.words("english")
stopwords.append(['so', 'just', 'maybe'])
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
data = load_data(csv_data_path)['content']
data.head()

0    Welcome to our very new community forum.  | “W...
1    Option1:   Sign up for an account at https://O...
2    Hi, I checked around, and I can’t seem to find...
3    Hi, First of all, thank you for providing such...
4    I came across the Mozilla Location Service her...
Name: content, dtype: object

# Content Preprocessing

In [14]:
def lemmatize(posts: pd.DataFrame, allowed_post_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm" if small_model else "en_core_web_trf", disable=["parser", 'ner'])
    texts_out = []

    for post in posts:
        doc = nlp(post)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_post_tags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out

lemmatized_data = lemmatize(data.astype(str))
lemmatized_data

/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


['welcome very new community forum ’ good way begin say well introduce anonymous lead here exciting thing currently work say just want here as long ’re nice other',
 'option1 sign account directly download file option sign account sign dashboard use email token go download tab dashboard click download link download csv database edit ’re get rate_limtied error allow only download account day prevent abuse see error only able try again tomorrow use proper download manager/ browser',
 'check around seem find documentation different column datum mean ’m refer complete dataset available download gb e.g. range mcc sample let know thing document thank',
 'first thank provide such abundant amount datum worldwide curious way identify carrier use cell datum set download thank so much good',
 'come here say ’ build ’m confused dataset different related question ’m unclear use service identify device location doc indicate use',
 'know love know lot change change like other like have trouble improv

In [15]:
def tokenize_lemma(posts: list):
    final = []
    for post in posts:
        new = gensim.utils.simple_preprocess(post, deacc=True)
        final.append(new)
    return final

tokenized_data = tokenize_lemma(lemmatized_data)
data = [[word for word in phrase if word not in stopwords] for phrase in tokenized_data]
data

[['welcome',
  'new',
  'community',
  'forum',
  'good',
  'way',
  'begin',
  'say',
  'well',
  'introduce',
  'anonymous',
  'lead',
  'exciting',
  'thing',
  'currently',
  'work',
  'say',
  'want',
  'long',
  'nice'],
 ['option',
  'sign',
  'account',
  'directly',
  'download',
  'file',
  'option',
  'sign',
  'account',
  'sign',
  'dashboard',
  'use',
  'email',
  'token',
  'go',
  'download',
  'tab',
  'dashboard',
  'click',
  'download',
  'link',
  'download',
  'csv',
  'database',
  'edit',
  'get',
  'rate_limtied',
  'error',
  'allow',
  'download',
  'account',
  'day',
  'prevent',
  'abuse',
  'see',
  'error',
  'able',
  'try',
  'tomorrow',
  'use',
  'proper',
  'download',
  'manager',
  'browser'],
 ['check',
  'around',
  'seem',
  'find',
  'documentation',
  'different',
  'column',
  'datum',
  'mean',
  'refer',
  'complete',
  'dataset',
  'available',
  'download',
  'gb',
  'range',
  'mcc',
  'sample',
  'let',
  'know',
  'thing',
  'documen

# Generate Bigrams/Trigrams

In [16]:

bigrams_phrases = gensim.models.Phrases(data, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data], threshold=100)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return (bigram[doc] for doc in texts)

def make_trigrams(texts):
    return (trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data)
data_bigrams_trigrams = make_trigrams(data_bigrams)
bgtg = list(data_bigrams_trigrams)
bgtg

[['welcome',
  'new',
  'community',
  'forum',
  'good',
  'way',
  'begin',
  'say',
  'well',
  'introduce',
  'anonymous',
  'lead',
  'exciting',
  'thing',
  'currently',
  'work',
  'say',
  'want',
  'long',
  'nice'],
 ['option',
  'sign',
  'account',
  'directly',
  'download',
  'file',
  'option',
  'sign',
  'account',
  'sign',
  'dashboard',
  'use',
  'email',
  'token',
  'go',
  'download',
  'tab',
  'dashboard',
  'click',
  'download',
  'link',
  'download',
  'csv',
  'database',
  'edit',
  'get',
  'rate_limtied',
  'error',
  'allow',
  'download',
  'account',
  'day',
  'prevent',
  'abuse',
  'see',
  'error',
  'able',
  'try',
  'tomorrow',
  'use',
  'proper',
  'download',
  'manager',
  'browser'],
 ['check',
  'around',
  'seem',
  'find',
  'documentation',
  'different',
  'column',
  'datum',
  'mean',
  'refer',
  'complete',
  'dataset',
  'available',
  'download',
  'gb',
  'range',
  'mcc',
  'sample',
  'let',
  'know',
  'thing',
  'documen

# TF-IDF non-useful common word removal

In [17]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(bgtg)
corpus = [id2word.doc2bow(text) for text in bgtg]

corpus

tfidf = TfidfModel(corpus=corpus, id2word=id2word, )

low_value = 0.03
words = []
words_missing_tfidf = []
for iter in range(0,len(corpus)):
    bow = corpus[iter]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_tfidf]
    corpus[iter] = new_bow

# Generate id2word dictionary

In [18]:
# Deprecated since TF-IDF handles this already
# id2word = corpora.Dictionary(bgtg)

# corpus = []

# for text in bgtg:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# corpus

# LDA Visualization

In [19]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10,
random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [20]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecat

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.228253  0.114476       1        1  24.486115
6      0.090281  0.215006       2        1  17.575184
3     -0.079717 -0.012064       3        1  12.943278
1     -0.059931  0.190818       4        1  10.684227
4      0.100095 -0.209264       5        1   8.584390
2     -0.232994  0.111289       6        1   7.762474
9     -0.083291 -0.226895       7        1   6.376535
0     -0.194903 -0.099722       8        1   4.493574
7      0.053283  0.004320       9        1   4.188042
8      0.178922 -0.087965      10        1   2.906181, topic_info=         Term       Freq      Total Category  logprob  loglift
34       file  26.000000  26.000000  Default  30.0000  30.0000
25        csv  23.000000  23.000000  Default  29.0000  29.0000
30   download  34.000000  34.000000  Default  28.0000  28.0000
74       cell  99.000000  99.000000  Default  27.0000  27.0000
304      area  29.000000  29.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
18       work   1.292863  16.133046  Topic10  -4.6903   1.0143
137      help   1.350302  37.864299  Topic10  -4.6468   0.2046
703      miss   0.916217   3.804223  Topic10  -5.0347   2.1147
17       well   0.913200  10.745279  Topic10  -5.0380   1.0731
56      datum   0.944168  72.395829  Topic10  -5.0046  -0.8013

[575 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1218      6  0.543283  abcdef
19        1  0.086754    able
19        2  0.173507    able
19        3  0.086754    able
19        5  0.086754    able
...     ...       ...     ...
423       3  0.760352    year
423      10  0.168967    year
912       4  0.799260     yet
1090      1  0.881830    zero
233       1  0.786070     zip

[839 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 4, 2, 5, 3, 10, 1, 8, 9])